Imports

In [3]:
#imports
import sys
import os
import pandas as pd

utils_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'utils'))
if utils_path not in sys.path:
    sys.path.insert(0, utils_path)

from general_utils import load_data, prepare_all_samples
from naive_extractor_utils import naive_extraction

Data Loading

In [5]:
# Load data
df = load_data("../data/inference_dataset.csv")
print(f"Loaded {len(df)} records")

Loaded 101 records


In [7]:
#Inspect df
df.head()

,doc_id,note_text,entities_json,dates_json,relative_dates_json
0,0,Ultrasound (30nd Jun 2024): no significant fin...,"[{'id': 'ent_1', 'value': 'Ultrasound', 'cui':...","[{'id': 'abs_1', 'value': '30nd Jun 2024', 'st...",[]
1,1,Labs (27th Sep 2024): anemia. resolving Skin:...,"[{'id': 'ent_1', 'value': 'anemia', 'cui': 'C0...","[{'id': 'abs_1', 'value': '27th Sep 2024', 'st...",[]
2,2,URGENT REVIEW (2024-10-04): cough. suspect ost...,"[{'id': 'ent_1', 'value': 'REVIEW', 'cui': 'C1...","[{'id': 'abs_1', 'value': '2024-10-04', 'start...",[]
3,3,URGENT REVIEW (13rd Feb 2025) MRI of the brain...,"[{'id': 'ent_1', 'value': 'REVIEW', 'cui': 'C0...","[{'id': 'abs_1', 'value': '13rd Feb 2025', 'st...",[]
4,4,New pt((18/11/24)): pt presents with nausea/vo...,"[{'id': 'ent_1', 'value': 'nausea', 'cui': 'C0...","[{'id': 'abs_1', 'value': '18/11/24', 'start':...",[]


In [8]:
# Prepare all samples
samples = prepare_all_samples(df)
print(f"Prepared {len(samples)} samples")
#samples[2]

Prepared 101 samples


Naive Inference

In [10]:
#Define max distance (number of characters)
MAX_DISTANCE = 200

In [ ]:
#Make predictions
predictions = []

for sample in samples:
    # Combine absolute and relative dates
    all_dates = sample['dates'] + sample['relative_dates']
    
    # Run naive extraction on all dates
    relationships = naive_extraction(sample['entities_list'], all_dates, max_distance = MAX_DISTANCE)
    predictions.extend(relationships)

In [15]:
# Look at predictions
#predictions